## Data Types Check

In [38]:
import polars as pl

# Read the data from directory
df = pl.read_parquet("../0 - Data/1 - merge/merged_transactions.pq")
df.head()

User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
i64,i64,i64,i64,i64,str,str,str,i64,str,str,f64,i64,str,str,i64,str,str,i64,str,i64,str,i64,str,str,i64,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,f64,f64,str,str,str,i64,i64
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",0,"""Visa""","""Debit""",4344676511950444,"""12/2022""",623,"""YES""",2,"""$24295""","""09/2002""",2008,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",1,"""Visa""","""Debit""",4956965974959986,"""12/2020""",393,"""YES""",2,"""$21968""","""04/2014""",2014,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",2,"""Visa""","""Debit""",4582313478255491,"""02/2024""",719,"""YES""",2,"""$46414""","""07/2003""",2004,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",3,"""Visa""","""Credit""",4879494103069057,"""08/2024""",693,"""NO""",1,"""$12400""","""01/2003""",2012,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",4,"""Mastercard""","""Debit (Prepaid)""",5722874738736011,"""03/2009""",75,"""YES""",1,"""$28""","""09/2008""",2009,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5


#### Combine `Year`, `Month`, `Day`, and `Time` into a single `Datetime` column

In [39]:
df = df.with_columns([
    # Ensure 'Year', 'Month', 'Day' are numeric first
    pl.col('Year').cast(pl.Int32).alias('Year'),
    pl.col('Month').cast(pl.Int32).alias('Month'),
    pl.col('Day').cast(pl.Int32).alias('Day'),
    
    # Ensure 'Time' is string for concatenation
    pl.col('Time').cast(pl.Utf8).alias('Time'),

    # Create a new 'Datetime_str' column by concatenating 'Year', 'Month', 'Day', and 'Time' as a string
    pl.concat_str(
        [pl.col("Year").cast(pl.Utf8), 
         pl.col("Month").cast(pl.Utf8).str.zfill(2), 
         pl.col("Day").cast(pl.Utf8).str.zfill(2), 
         pl.col("Time")],
        separator="-"
    ).alias("Datetime_str")
])

# Parse the concatenated 'Datetime_str' into a proper Datetime column
df = df.with_columns([
    pl.col("Datetime_str").str.strptime(pl.Datetime, format="%Y-%m-%d-%H:%M").alias("Datetime")
])

# Drop unnecessary columns: Time and temporary Datetime_str
df = df.drop(["Time", "Datetime_str"])

#### Handle Rest of the Date Columns

- Expire Column
- Acc Open Date

In [40]:
# Convert to date
df = df.with_columns(
        (
            pl.col("Expires")
            .str.strptime(pl.Date, format="%m/%Y", strict=False)
            .alias("Expires")
        )
    )

# Convert to date
df = df.with_columns(
        (
            pl.col("Acct Open Date")
            .str.strptime(pl.Date, format="%m/%Y", strict=False)
            .alias("Acct Open Date")
        )
    )

#### Handle Financial Columns

- Amount
- Credit Limit
- Yearly Incom - Person
- Total Debt
- Per Capita Income - Zipcode

In [41]:
import polars as pl

# Define the financial columns that need cleaning
financial_columns = [
    "Amount", "Credit Limit", "Yearly Income - Person", 
    "Total Debt", "Per Capita Income - Zipcode"
]

# Apply the transformations in a loop
df = df.with_columns([
    # Clean each financial column in the list
    pl.when(pl.col(col).is_not_null())
      .then(pl.col(col).str.replace('$', '', literal=True))
      .otherwise(None)
      .cast(pl.Float64)
      .alias(col)
    for col in financial_columns
] + [
    # Cast other columns directly
    pl.col("FICO Score").cast(pl.Int64).alias("FICO Score"),
    pl.col("Num Credit Cards").cast(pl.Int64).alias("Num Credit Cards")
])

#### Handle Boolean Columns

- Is Fraud?
- Has Chip
- Card on Dark Web

In [42]:
# Convert columns 'Is Fraud?', 'Has Chip', 'Card on Dark Web' to boolean
df = df.with_columns([
    pl.when(pl.col("Is Fraud?") == "Yes").then(1).otherwise(0).alias("Is Fraud"),
    pl.when(pl.col("Has Chip") == "YES").then(1).otherwise(0).alias("Has Chip"),
    pl.when(pl.col("Card on Dark Web") == "Yes").then(1).otherwise(0).alias("Card on Dark Web")
])

df = df.drop([ "Is Fraud?" ])

#### Categorical Columns (UTF8)

- Merchant Name
- Card Type
- Card Brand
- Merchant City
- Merchant State
- Zip
- Use Chip

In [43]:
# Ensure categorical or string columns are UTF8 type
df = df.with_columns([
    pl.col("Merchant Name").cast(pl.Utf8).alias("Merchant Name"),
    pl.col("Card Brand").cast(pl.Utf8).alias("Card Brand"),
    pl.col("Card Type").cast(pl.Utf8).alias("Card Type"),
    pl.col("Merchant City").cast(pl.Utf8).alias("Merchant City"),
    pl.col("Merchant State").cast(pl.Utf8).alias("Merchant State"),
    pl.col("Zip").cast(pl.Utf8).alias("Zip"),
    pl.col("Use Chip").cast(pl.Utf8).alias("Use Chip")
])

#### Handle Rest of the Numerical Columns (Float)

In [44]:
# Ensure 'Latitude' and 'Longitude' are float type
df = df.with_columns([
    pl.col("Latitude").cast(pl.Float64).alias("Latitude"),
    pl.col("Longitude").cast(pl.Float64).alias("Longitude")
])

#### Handle Null Values

In [45]:
# Find all the nulls within the data
def check_for_null_cols(df: pl.DataFrame):
    null_counts = df.null_count()
    null_cols = []
    for idx, col in enumerate(null_counts):
        if col[0] > 0:
            null_cols.append(null_counts.columns[idx])

    return null_cols

print("Columns with NULL values:", check_for_null_cols(df))

Columns with NULL values: ['Merchant State', 'Zip', 'Errors?', 'Apartment']


In [46]:
# Fill null values with appropriate defaults
df = df.with_columns([
    pl.col("Zip").fill_null("Unknown"),
    pl.col("Errors?").fill_null(0),
    pl.col("Merchant State").fill_null("Unknown"),
    pl.col("Apartment").fill_null("Unknown")
])

In [47]:
# Just to be sure
print("Columns with NULL values:", check_for_null_cols(df))

Columns with NULL values: []


#### Type Checked Data

In [48]:
df.head()

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Datetime,Is Fraud
i64,i64,i32,i32,i32,f64,str,str,str,str,str,i64,str,i64,str,str,i64,date,i64,i32,i64,f64,date,i64,i32,str,i64,i64,i64,i64,str,str,str,str,str,i64,f64,f64,f64,f64,f64,i64,i64,datetime[μs],i32
0,0,2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""",0,"""Visa""","""Debit""",4344676511950444,2022-12-01,623,1,2,24295.0,2002-09-01,2008,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,29278.0,59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""",1,"""Visa""","""Debit""",4956965974959986,2020-12-01,393,1,2,21968.0,2014-04-01,2014,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,29278.0,59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""",2,"""Visa""","""Debit""",4582313478255491,2024-02-01,719,1,2,46414.0,2003-07-01,2004,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,29278.0,59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""",3,"""Visa""","""Credit""",4879494103069057,2024-08-01,693,0,1,12400.0,2003-01-01,2012,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,29278.0,59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""",4,"""Mastercard""","""Debit (Prepaid)""",5722874738736011,2009-03-01,75,1,1,28.0,2008-09-01,2009,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,29278.0,59696.0,127613.0,787,5,2018-01-02 06:28:00,0


#### Save the cleaned data

In [49]:
# Type checked data but with full columns (For further data exploration)
df.write_parquet("../0 - Data/2 - clean/clean_transactions.pq")